In [1]:
import pymatgen as mg
import pandas as pd
import numpy as np
import os
from pymatgen.util.coord import get_angle

"""The function extracts coordinates of atoms from a structure dictionary (which is obtained from pymatgen structure object for a given compound) 
and makes a dataframe with these data
"""

def coordinates_df(df_raw_coordinates):
    df = pd.DataFrame(index=range(0, df_raw_coordinates.shape[0]), columns = ['element', 'label', 'x', 'y', 'z'])
    
    for i in range(0, df.shape[0]):
        df['x'][i] = df_raw_coordinates['xyz'][i][0]
        df['y'][i] = df_raw_coordinates['xyz'][i][1]
        df['z'][i] = df_raw_coordinates['xyz'][i][2]
        df['element'][i] = df_raw_coordinates['species'][i][0]['element']
        df['label'][i] = df_raw_coordinates['label'][i]
    
    return df


# Generates geometric features and adds them into the dictionary features_dict
def features_geom(df_coordinates, features_dict, number_of_elements, structure_dict):
    
    # Lattice parameters
    features_dict['lattice_a'] = structure_dict['lattice']['a']
    features_dict['lattice_c'] = structure_dict['lattice']['c']
    
    """Here we define dataframes with coordinates for pnictogens, for Fe-like atoms (those which form trilayer structure together with pnictogens)
    and for the atoms which form a layer between Fe-Pn trilayer structure (in what follows we call it K-layer)"""

    elements = []
    for el in df_coordinates['element']:
        elements.append(el)
    
    # For Fe-like elements
    if 'Fe' in elements:
        df_Fe = df_coordinates[df_coordinates['element']=='Fe']
    if 'Ni' in elements:
        df_Fe = df_coordinates[df_coordinates['element']=='Ni']
    if 'Co' in elements:
        df_Fe = df_coordinates[df_coordinates['element']=='Co']
    
    # For pnictogens (and chalcogens) 
    if 'As' in elements:
        df_Pn = df_coordinates[df_coordinates['element']=='As']
    if 'Se' in elements:
        df_Pn = df_coordinates[df_coordinates['element']=='Se']
    if 'P' in elements:
        df_Pn = df_coordinates[df_coordinates['element']=='P']
    
    # For the atoms between trilayers
    if number_of_elements == 3:
        if 'K' in elements:
            df_K = df_coordinates[df_coordinates['element']=='K']
        if 'Ca' in elements:
            df_K = df_coordinates[df_coordinates['element']=='Ca']
        if 'Sr' in elements:
            df_K = df_coordinates[df_coordinates['element']=='Sr'] 
        if 'Ba' in elements:
            df_K = df_coordinates[df_coordinates['element']=='Ba'] 
        if 'Eu' in elements:
            df_K = df_coordinates[df_coordinates['element']=='Eu'] 
    
     
    """This piece of code is to find the angle between Pn_Fe_Pn and the distance between Fe and Pn layers 
    (see the picture for clarification)"""
    
    # Vector for a Fe atom 
    vector_Fe = []
    vector_Fe.extend([df_Fe['x'][df_Fe.index[0]], df_Fe['y'][df_Fe.index[0]], df_Fe['z'][df_Fe.index[0]]])
    vector_Fe = np.asarray(vector_Fe, dtype=np.float32)
    
    # List of vectors for pnictogens
    vectors_Pn = []
    # List of distances for the Fe atom and pnictogen atoms
    distances_Fe_Pn = []
    for i in range(0, df_Pn['element'].shape[0]):
        vectors_Pn.extend([[df_Pn['x'][df_Pn.index[i]], df_Pn['y'][df_Pn.index[i]], df_Pn['z'][df_Pn.index[i]]]])
        vectors_Pn[i] = np.asarray(vectors_Pn[i], dtype=np.float32)
        distance = np.linalg.norm(vectors_Pn[i] - vector_Fe)
        distances_Fe_Pn.append(distance)
    
    min_distance = min(distances_Fe_Pn)
    # Define list of vectors (there are always two of them) for which distance to the Fe atom is min 
    vectors_Pn_min = []
    for i in range(0, len(distances_Fe_Pn)):
        if abs(distances_Fe_Pn[i] - min_distance)<0.00001:
             vectors_Pn_min.append(vector_Fe  - vectors_Pn[i])            
    
    # Add the angle between Pn_Fe_Pn to the dictionary
    features_dict['angle_Pn_Fe_Pn'] = get_angle(vectors_Pn_min[0], vectors_Pn_min[1])
    # Add the distance between Fe and Pn layers
    features_dict['distance_Fe_Pn'] = min_distance
    
    
    '''Find distance between Fe layers'''
    
    layers_distances = []
    for j in range(0, df_Fe.shape[0]):
            if df_Fe.index[j]!=df_Fe.index[0]:
                if (abs(df_Fe['x'][df_Fe.index[j]] - df_Fe['x'][df_Fe.index[0]])<0.00001) \
                   and (abs(df_Fe['y'][df_Fe.index[j]] - df_Fe['y'][df_Fe.index[0]])<0.00001) \
                   and (abs(df_Fe['z'][df_Fe.index[j]] - df_Fe['z'][df_Fe.index[0]])>0.1):
                       features_dict['Fe_layers_distance'] = abs(df_Fe['z'][df_Fe.index[j]] - df_Fe['z'][df_Fe.index[0]])
                        
    
    """Distance between the K-layer and the pnictogen layer"""
    
    if number_of_elements == 3:
        vector_K = []
        distances_K_Pn = []
        vector_K.extend([df_K['x'][df_K.index[1]], df_K['y'][df_K.index[1]], df_K['z'][df_K.index[1]]])
        
        for i in range(0, df_K['element'].shape[0]):
            distance = abs(np.linalg.norm(vectors_Pn[i] - vector_K))
            distances_K_Pn.append(distance)
        min_distance = min(distances_K_Pn)
        features_dict['distance_Pn_K'] = min_distance
    
    return features_dict

# Takes features_dict and generates geometric features for a given compound
def generate_features_list(features_dict):
    
    features_to_append = []
    
    # Append compound
    features_to_append.append(features_dict['compound'])
    features_to_append.append(features_dict['ICSD number'])
    features_to_append.append(features_dict['number_of_elements'])
        
    # Add geometric features for a given compound to the features_dict
    features_to_append.append(features_dict['distance_Fe_Pn'])
    features_to_append.append(features_dict['angle_Pn_Fe_Pn'])
    features_to_append.append(features_dict['Fe_layers_distance'])
    features_to_append.append(features_dict['distance_Pn_K'])
    features_to_append.append(features_dict['lattice_a'])
    features_to_append.append(features_dict['lattice_c'])
    
    return features_to_append



# Path to the directory where iron pnictides .cif files are located 
cif_path = '/home/dima/Desktop/ML projects/Pnicktides/Features/data/'

cif_files = os.listdir(cif_path)
df_features = pd.DataFrame(columns = ['compound', 'ICSD number', 'number_of_elements', 'distance_Fe_Pn', 'angle_Pn_Fe_Pn', 
                                                                       'Fe_layers_distance', 'distance_Pn_K', 'lattice_a', 'lattice_c'])


####################################################################################
####################################################################################


for file_name in cif_files:
    
    features_dict = {'compound': '', 'ICSD number': '', 'number_of_elements': '', 'distance_Fe_Pn': '', 'angle_Pn_Fe_Pn': '', 
                     'Fe_layers_distance': '', 'distance_Pn_K': '', 'lattice_a': '', 'lattice_c': ''}
    
    
    """First, add the nongeometric features and labels into the df_features dataframe"""
    
    # Add ICSD number into features_dict
    features_dict['ICSD number'] = file_name.replace('ICSD_CollCode', '').replace('.cif', '')
    
    # Define pymatgen structure object
    structure = mg.Structure.from_file(cif_path+file_name)
    # Add compound name into features_dict
    features_dict['compound'] = structure.composition.reduced_formula
    # Count number of elements as the number of upper letters in the compound name
    number_of_elements = 0
    for symbol in str(features_dict['compound']):
        if str.isupper(str(symbol)):
            number_of_elements+=1
    features_dict['number_of_elements'] = number_of_elements
    
    
    """Add geometric features into df_features"""
    
    structure_dict = structure.as_dict()
    # Generate ccordinates dataframe for a given compound
    df_raw_coordinates = pd.DataFrame.from_dict(structure_dict['sites'])    
    df_coordinates = coordinates_df(df_raw_coordinates)
    
    # Generate geometric features into the features_dict
    features_dict = features_geom(df_coordinates, features_dict, number_of_elements = number_of_elements, structure_dict = structure_dict)
     
    # Generate features list
    features_list = generate_features_list(features_dict)
    
    # Add features_list as a row to the df_features dataset
    df_features.loc[len(df_features)] = features_list

df_features

,compound,ICSD number,number_of_elements,distance_Fe_Pn,angle_Pn_Fe_Pn,Fe_layers_distance,distance_Pn_K,lattice_a,lattice_c
0,Ba(NiP)2,85408,3,2.259574,103.720261,5.91,3.35094,3.94700,11.8200
1,Ca2Co3.72As4,236860,3,2.325001,105.278093,5.1399,3.1396,3.99060,10.2798
2,Eu(FeAs)2,169687,3,2.381615,108.911100,6.026,3.22702,3.91600,12.0520
3,Ba(FeP)2,169745,3,2.261028,106.116599,6.211,3.32423,3.84350,12.4220
4,Ca(FeAs)2,192310,3,2.373415,109.121072,5.84,3.16456,3.89250,11.6800
5,Sr(FeAs)2,182330,3,2.388827,108.889382,6.1586,3.26755,3.92890,12.3172
6,K(FeAs)2,31473,3,2.389305,110.706738,6.9305,3.40007,3.84200,13.8610
7,LaFeAsO,173432,4,2.411983,107.471969,,,4.03533,8.7409
8,Ba(NiAs)2,164197,3,2.360736,103.202419,5.8095,3.42879,4.14740,11.6190
